In [1]:
# source: https://www.kaggle.com/lewisfogden/the-life-modelling-problem

import modelx as mx

space = mx.new_space("sample")

In [2]:
space.q = {
    0: 0.001,
    1: 0.002,
    2: 0.003,
    3: 0.003,
    4: 0.004,
    5: 0.004,
    6: 0.005,
    7: 0.007,
    8: 0.009,
    9: 0.011
}

space.w = {
    0: 0.05,
    1: 0.07,
    2: 0.08,
    3: 0.10,
    4: 0.14,
    5: 0.20,
    6: 0.20,
    7: 0.20,
    8: 0.10,
    9: 0.04
}

space.P = 100
space.S = 25000
space.T = 10

In [3]:
@mx.defcells
def num_in_force(t):
    if t == 0:
        return 1
    elif t >= T:
        return 0
    else:
        return num_in_force(t-1) - num_deaths(t-1) - num_lapses(t-1)

@mx.defcells
def num_deaths(t):
    if t < T:
        return num_in_force(t) * q[t]
    else:
        return 0

@mx.defcells
def num_lapses(t):
    if t < T:
        return num_in_force(t) * w[t]
    else:
        return 0

@mx.defcells
def claims(t):
    return num_deaths(t) * S

@mx.defcells
def premiums(t):
    return num_in_force(t) * P

@mx.defcells
def net_cashflow(t):
    return premiums(t) - claims(t)

In [4]:

def npv(cashflow, term, int_rate):
    value = 0
    for t in range(0, term):
        value = (value + cashflow(t)) / (1 + int_rate)
    return value


In [5]:
%timeit npv(net_cashflow, 10, 0.02)

71 µs ± 3.03 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
npv(net_cashflow, 10, 0.02)

13.510102735717568

## Vectorization

In [7]:
import numpy as np

space.np = np

In [8]:
@mx.defcells
def num_in_force(t):
    if t == 0:
        return np.ones(10000)
    elif t >= T:
        return np.zeros(10000)
    else:
        return num_in_force(t-1) - num_deaths(t-1) - num_lapses(t-1)

In [9]:
npv(net_cashflow, 10, 0.02)

array([13.51010274, 13.51010274, 13.51010274, ..., 13.51010274,
       13.51010274, 13.51010274])

## Varying terms

In [14]:
last_t = np.full(shape=10000, fill_value=10)

last_t[1] = 5    # second policy

last_t

array([10,  5, 10, ..., 10, 10, 10])

In [19]:
space.last_t = last_t

In [24]:
@mx.defcells
def num_maturities(t):
    return (t == last_t) * (num_in_force(t-1) - num_deaths(t-1) - num_lapses(t-1))

In [22]:
num_maturities(5)

array([0.        , 0.62008287, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [25]:
@mx.defcells
def num_in_force(t):
    if t == 0:
        return np.ones(10000)
    elif t >= T:
        return np.zeros(10000)
    else:
        return num_in_force(t-1) - num_deaths(t-1) - num_lapses(t-1) - num_maturities(t)

In [61]:
num_in_force(5)

array([0.62008287, 0.        , 0.62008287, ..., 0.62008287, 0.62008287,
       0.62008287])

In [72]:
net_cashflow(6)

array([-12.33964906,   0.        , -12.33964906, ..., -12.33964906,
       -12.33964906, -12.33964906])

In [70]:
npv(net_cashflow, 10, 0.02)

array([ 13.51010274, 137.59737921,  13.51010274, ...,  13.51010274,
        13.51010274,  13.51010274])